# Visualizing m1 versus m3 traffic data with CesiumJS

In [1]:
import pandas as pd
import glob
from CZML_builder import czml

#### <br> so6 file parser

In [2]:
def df(path):
    columns = ['segment_identifier', 'flight_origin', 'flight_destination', 'aircraft_type', 
           'time_begin', 'time_end', 'fl_begin', 'fl_end', 'status', 'callsign', 
           'date_begin', 'date_end', 'lat_begin', 'lon_begin', 'lat_end', 'lon_end',
           'flight_identifier', 'sequence', 'length', 'parity']
    parser = lambda x,y: pd.datetime.strptime(x + y, '%y%m%d%H%M%S')

    df = pd.read_csv(path, sep=' ', header=None, names=columns, 
                     parse_dates={'datetime_begin': ['date_begin','time_begin'], 
                              'datetime_end': ['date_end','time_end']}, date_parser=parser,
                    dtype={'aircraft_type': str})
    df.query('length > 0', inplace=True) # filter null segments
    df['duration'] = (df['datetime_end'] - df['datetime_begin']).dt.total_seconds()
    df['alt_begin (m)'] = df['fl_begin'].apply(lambda x: x * 30.48) # FL to meters
    lat_lon_cols = ['lat_begin','lon_begin','lat_end','lon_end']
    df[lat_lon_cols] = df[lat_lon_cols].apply(lambda x: x / 60)
    return df

#### <br> Parse separately m1 and m3 so6 files

In [ ]:
df1 = df('your_m1_file.so6')
df3 = df('your_m3_file.so6')

#### <br> Apply some filters if needed

In [6]:
df1 = df1.query('aircraft_type=="A388" and flight_origin=="LFPG"')
df3 = df3.query('aircraft_type=="A388" and flight_origin=="LFPG"')

#### <br> Create one CZML file with both m1 and m3 tracks/trajectories

In [7]:
czml([df1, df3], "m1vsm3.czml", path=True, point=True)